In [3]:
using MLBase
using LinearAlgebra
using MultivariateStats

### Data Preprocessing Utilities

#### Data Repetition

In [2]:
println(repeach(1:3,2));
println(repeach(1:3,[3,2,1]))

[1, 1, 2, 2, 3, 3]
[1, 1, 1, 2, 2, 3]


In [17]:
repeachcol(rand(2,3),[3,2])   #Repeat each column in matrix a for n times. Here n can be either a scalar or a vector with 

2×5 Array{Float64,2}:
 0.94668   0.94668   0.94668   0.802813  0.802813
 0.465706  0.465706  0.465706  0.560968  0.560968

In [19]:
repeachrow(rand(2,3),[2,3])  #Repeat each row in matrix a for n times. Here n can be either a scalar or a vector with 

5×3 Array{Float64,2}:
 0.734464  0.222528  0.544563
 0.734464  0.222528  0.544563
 0.607427  0.955209  0.75498 
 0.607427  0.955209  0.75498 
 0.607427  0.955209  0.75498 

### Label Processing

In machine learning, we often need to first attach each class with an integer label. This package provides a type LabelMap that captures the association between discrete values (e.g a finite set of strings) and integer labels.

Together with LabelMap, the package also provides a function labelmap to construct the map from a sequence of discrete values, and a function labelencode to map discrete values to integer labels.

In [20]:
lm=labelmap(["a","a","b","b","c"])

LabelMap (with 3 labels):
[1] a
[2] b
[3] c


In [22]:
labelencode(lm,"b")

2

In [23]:
labelencode(lm,["a","b","a"])

3-element Array{Int64,1}:
 1
 2
 1

Note that labelencode can be applied to either single value or an array.

The package also provides a function groupindices to group indices based on associated labels.

In [24]:
groupindices(3,[1,1,1,2,2,3,2])

3-element Array{Array{Int64,1},1}:
 [1, 2, 3]
 [4, 5, 7]
 [6]      

In [25]:
groupindices(lm,["a","a","c","b","b"])

3-element Array{Array{Int64,1},1}:
 [1, 2]
 [4, 5]
 [3]   

### Classification

A classification procedure, no matter how sophisticated it is, generally consists of two steps: (1) assign a score/distance to each class, and (2) choose the class that yields the highest score/lowest distance.



This package provides a function classify and its friends to accomplish the second step, that is, to predict labels based on scores.

Classify based on scores given in x and the order of scores specified in ord.

Generally, ord can be any instance of type Ordering. However, it usually enough to use either Forward or Reverse:

* ord = Forward: higher value indicates better match (e.g., similarity)
* ord = Reverse: lower value indicates better match (e.g., distances)
When ord is omitted, it is defaulted to Forward.

When x is a vector, it produces an integer label. When x is a matrix, it produces a vector of integers, each for a column of x.

In [28]:
println(classify([0.2,0.5,0.3]))
println(classify([0.2,0.5,0.3],Forward))
println(classify([0.2,0.5,0.3],Reverse))

2
2
1


In [30]:
println(classify([0.2 0.5 0.3; 0.7 0.6 0.2]')) # --> [2, 1]
println(classify([0.2 0.5 0.3; 0.7 0.6 0.2]', Forward)) # --> [2, 1]
println(classify([0.2 0.5 0.3; 0.7 0.6 0.2]', Reverse)) # --> [1, 3]

[2, 1]
[2, 1]
[1, 3]


### Performance Evaluation

This package provides tools to assess the performance of a machine learning algorithm.

#### Classification Performance

** correctrate(gt, pred) **
  
  Compute correct rate of predictions given by pred w.r.t. the ground truths given in gt.

** errorrate(gt, pred) **
  
  Compute error rate of predictions given by pred w.r.t. the ground truths given in gt.

** confusmat(k, gt, pred) **
  
  Compute the confusion matrix of the predictions given by pred w.r.t. the ground truths given in gt. Here, k is the number of classes.

It returns an integer matrix R of size (k, k), such that R(i, j) == countnz((gt .== i) & (pred .== j)).

In [31]:
gt=[1, 1, 1, 2, 2, 2, 3, 3];
pred = [1, 1, 2, 2, 2, 3, 3, 3];

In [35]:
C=confusmat(3,gt,pred);
C./sum(C,2)

┌ Warning: `sum(a::AbstractArray, dims)` is deprecated, use `sum(a, dims=dims)` instead.
│   caller = top-level scope at In[35]:2
└ @ Core In[35]:2


3×3 Array{Float64,2}:
 0.666667  0.333333  0.0     
 0.0       0.666667  0.333333
 0.0       0.0       1.0     

In [37]:
println(trace(C) / length(gt)) # compute correct rate from confusion matrix
correctrate(gt, pred)

0.75


Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38


0.75

#### Hit rate (for retrieval tasks)
    hitrate(gt, ranklist, k)

Compute the hitrate of rank k for a ranked list of predictions given by ranklist w.r.t. the ground truths given in gt.

Particularly, if gt[i] is contained in ranklist[1:k, i], then the prediction for the i-th sample is said to be hit within rank ``k``. The hitrate of rank k is the fraction of predictions that hit within rank k.

    hitrates(gt, ranklist, ks)

Compute hit-rates of multiple ranks (as given by a vector ks). It returns a vector of hitrates r, where r[i] corresponding to the rank ks[i].

Note that computing hit-rates for multiple ranks jointly is more efficient than computing them separately.

Receiver Operating Characteristics (ROC)
Receiver Operating Characteristics (ROC) is often used to measure the performance of a detector, thresholded classifier, or a verification algorithm.

##### The ROC Type
This package uses an immutable type ROCNums defined below to capture the ROC of an experiment:

    immutable ROCNums{T<:Real}
        p::T    # positive in ground-truth
        n::T    # negative in ground-truth
        tp::T   # correct positive prediction
        tn::T   # correct negative prediction
        fp::T   # (incorrect) positive prediction when ground-truth is negative
        fn::T   # (incorrect) negative prediction when ground-truth is positive
    end
One can compute a variety of performance measurements from an instance of ROCNums (say r):

    true_positive(r)
the number of true positives (r.tp)

    true_negative(r)
the number of true negatives (r.tn)

    false_positive(r)
the number of false positives (r.fp)

    false_negative(r)
the number of false negatives (r.fn)

true_postive_rate(r)
the fraction of positive samples correctly predicted as positive, defined as r.tp / r.p

    true_negative_rate(r)
the fraction of negative samples correctly predicted as negative, defined as r.tn / r.n

    false_positive_rate(r)
the fraction of negative samples incorrectly predicted as positive, defined as r.fp / r.n

    false_negative_rate(r)
the fraction of positive samples incorrectly predicted as negative, defined as r.fn / r.p

    recall(r)
Equivalent to true_positive_rate(r).

    precision(r)
the fraction of positive predictions that are correct, defined as r.tp / (r.tp + r.fp).

    f1score(r)
the harmonic mean of recall(r) and precision(r).

    Computing ROC Curves
The package provides a function roc to compute an instance of ROCNums or a sequence of such instances from predictions.

    roc(gt, pred)
Compute an ROC instance based on ground-truths given in gt and predictions given in pred.

    roc(gt, scores, thres[, ord])
Compute an ROC instance or an ROC curve (a vector of ROC instances), based on given scores and a threshold thres.

Prediction will be made as follows:

When ord = Forward: predicts 1 when scores[i] >= thres otherwise 0.
When ord = Reverse: predicts 1 when scores[i] <= thres otherwise 0.
When ord is omitted, it is defaulted to Forward.

Returns:

When thres is a single number, it produces a single ROCNums instance;
When thres is a vector, it produces a vector of ROCNums instances.
Note: Jointly evaluating an ROC curve for multiple thresholds is generally much faster than evaluating for them individually.

    roc(gt, (preds, scores), thres[, ord])
Compute an ROC instance or an ROC curve (a vector of ROC instances) for multi-class classification, based on given predictions, scores and a threshold thres.

Prediction is made as follows:

When ord = Forward: predicts preds[i] when scores[i] >= thres otherwise 0.
When ord = Reverse: predicts preds[i] when scores[i] <= thres otherwise 0.
When ord is omitted, it is defaulted to Forward.

Returns:

When thres is a single number, it produces a single ROCNums instance.
When thres is a vector, it produces an ROC curve (a vector of ROCNums instances).
Note: Jointly evaluating an ROC curve for multiple thresholds is generally much faster than evaluating for them individually.

    roc(gt, scores, n[, ord])
Compute an ROC curve (a vector of ROC instances), with respect to n evenly spaced thresholds from minimum(scores) and maximum(scores). (See above for details)

    roc(gt, (preds, scores), n[, ord])
Compute an ROC curve (a vector of ROC instances) for multi-class classification, with respect to n evenly spaced thresholds from minimum(scores) and maximum(scores). (See above for details)

    roc(gt, scores, ord])
Equivalent to roc(gt, scores, 100, ord).

    roc(gt, (preds, scores), ord])
Equivalent to roc(gt, (preds, scores), 100, ord).

roc(gt, scores)
Equivalent to roc(gt, scores, 100, Forward).

    roc(gt, (preds, scores))
Equivalent to roc(gt, (preds, scores), 100, Forward).

### Cross Validation

This package implements several cross validation schemes: Kfold, LOOCV, and RandomSub. Each scheme is an iterable object, of which each element is a vector of indices (indices of samples selected for training).

In [41]:
println(collect(Kfold(10,3)))
println(collect(StratifiedKfold([:a, :a, :a, :b, :b, :c, :c, :a, :b, :c], 3)))

Any[[1, 2, 3, 4, 5, 7, 8], [1, 4, 5, 6, 9, 10], [2, 3, 6, 7, 8, 9, 10]]
Any[[1, 2, 3, 4, 6, 9, 10], [2, 4, 5, 7, 8, 10], [1, 3, 5, 6, 7, 8, 9]]


In [42]:
println(collect(LOOCV(4)))

Any[[2, 3, 4], [1, 3, 4], [1, 2, 4], [1, 2, 3]]


#### Cross Validation Function

In [81]:
compute_center(X::Matrix{Float64}) = vec(mean(X, 2))

compute_rmse(c::Vector{Float64}, X::Matrix{Float64}) =
    sqrt(mean(sum((X .- c).^2,1)))

compute_rmse (generic function with 1 method)

In [82]:
const n = 200;
const data = [2., 3.] .+ randn(2, n);
size(data)

(2, 200)

In [84]:
# cross validation
scores = cross_validate(
    inds -> compute_center(data[:, inds]),        # training function
    (c, inds) -> compute_rmse(c, data[:, inds]),  # evaluation function
    n,              # total number of samples
    Kfold(n, 5))    # cross validation plan: 5-fold

# get the mean and std of the scores
(m, s) = mean_and_std(scores)

Add `using Statistics` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
in #27 at In[84]
┌ Warning: `mean(A::AbstractArray, dims)` is deprecated, use `mean(A, dims=dims)` instead.
│   caller = compute_center at In[81]:1 [inlined]
└ @ Core ./In[81]:1


(1.3726439255219731, 0.08732927326851335)

In [85]:
scores

5-element Array{Float64,1}:
 1.3527808936839616
 1.46673009394385  
 1.3403256129307248
 1.4500184688600166
 1.2533645581913124

### Model Tuning

Many machine learning algorithms and models come with design parameters that need to be set in advance. A widely adopted pratice is to search the parameters (usually through brute-force loops) that yields the best performance over a validation set. The package provides functions to facilitate this.

In [4]:
n_tr=20
n_te=10
d=5

theta=randn(d)
X_tr=randn(n_tr,d)
y_tr=X_tr*theta+0.1*randn(n_tr)
X_te=randn(n_te,d)
y_te=X_te*theta+0.1*randn(n_te)

10-element Array{Float64,1}:
  2.226227661011305    
  0.0007756233547431268
 -3.4052641342552734   
 -4.26789317180962     
  3.8946387331021515   
  4.387748145309897    
  2.014636518685055    
  4.782302415268727    
 -1.6258564204035715   
 -3.1582513985254472   

┌ Warning: Deprecated syntax `bias?`.
│ Use `bias ?` instead.
└ @ nothing none:3
┌ Warning: Deprecated syntax `bias ?'('`.
│ Use `bias ? '('` instead.
└ @ nothing none:3
┌ Warning: Deprecated syntax `bias ? (s[1:(end - 1)], s[end]):`.
│ Use `bias ? (s[1:(end - 1)], s[end]) :` instead.
└ @ nothing none:3
┌ Warning: Deprecated syntax `bias ? (s[1:(end - 1)], s[end]) :(`.
│ Use `bias ? (s[1:(end - 1)], s[end]) : (` instead.
└ @ nothing none:3
┌ Warning: Deprecated syntax `bias?`.
│ Use `bias ?` instead.
└ @ nothing none:2
┌ Warning: Deprecated syntax `bias ?'('`.
│ Use `bias ? '('` instead.
└ @ nothing none:2
┌ Warning: Deprecated syntax `bias ? (s[1:(end - 1)], s[end]):`.
│ Use `bias ? (s[1:(end - 1)], s[end]) :` instead.
└ @ nothing none:2
┌ Warning: Deprecated syntax `bias ? (s[1:(end - 1)], s[end]) :(`.
│ Use `bias ? (s[1:(end - 1)], s[end]) : (` instead.
└ @ nothing none:2
┌ Warning: Deprecated syntax `bias?`.
│ Use `bias ?` instead.
└ @ nothing none:3
┌ Warning: Deprecated syntax `b

#### tune the model

In [9]:
n_tr = 20  # number of training samples
n_te = 10  # number of testing samples
d = 5      # dimension of observations

theta = randn(d)
X_tr = randn(n_tr, d)
y_tr = X_tr * theta + 0.1 * randn(n_tr)
X_te = randn(n_te, d)
y_te = X_te * theta + 0.1 * randn(n_te)

## tune the model

function estfun(regcoef, bias)
    s = ridge(X_tr, y_tr, regcoef; bias=bias)
    return bias ? (s[1:end-1], s[end]) : (s, 0.0)
end

evalfun(m) = msd(X_te * m[1] + m[2], y_te)

r = gridtune(estfun, evalfun,
            ("regcoef", [1.0e-3, 1.0e-2, 1.0e-1, 1.0]),
            ("bias", (true, false));
            ord=Reverse,    # smaller msd value indicates better model
            verbose=true)   # show progress information

best_model, best_cfg, best_score = r

## print results

a, b = best_model
println("Best model:")
println("  a = $(a')"),
println("  b = $b")
println("Best config: regcoef = $(best_cfg[1]), bias = $(best_cfg[2])")
println("Best score: $(best_score)")

Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using LinearAlgebra` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kerne

[regcoef=0.001, bias=true] => 0.00807470846846661
[regcoef=0.01, bias=true] => 0.008203697061131415
[regcoef=0.1, bias=true] => 0.010779005367060985
[regcoef=1.0, bias=true] => 0.12127062093956756
[regcoef=0.001, bias=false] => 0.00928425910227475
[regcoef=0.01, bias=false] => 0.009469972323693572
[regcoef=0.1, bias=false] => 0.01263230776712955
[regcoef=1.0, bias=false] => 0.13184169983986588
Best model:
  a = [-0.528908 3.14287 -0.632566 -1.05288 0.287858]
  b = 0.02059481292852652
Best config: regcoef = 0.001, bias = true
Best score: 0.00807470846846661
